In [4]:
import numpy as np 
import matplotlib.pyplot as plt
import functools
import random
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import svm

In [9]:
def XOR_PUF(challenge_vector, stage_delays, n_PUF, n_stages):
    feature_vectors = []
    results = []
    for k in range(n_PUF): 
        #print(challenge_vector)
        feature_vector = [] 
        for i in range(n_stages): 
            feature = 1 
            for j in range(i, n_stages): 
                feature = feature * pow(-1, challenge_vector[j])
            feature_vector.append(feature) 
        feature_vector.append(1)
        results.append(np.dot(stage_delays[k], feature_vector) > 0)
        feature_vectors.append(feature_vector)
        
    flat = []
    for sublist in feature_vectors:
        for element in sublist:
            flat.append(element)
    return flat, functools.reduce(lambda x, y: x ^ y, results)

In [23]:
def iPUF(upper_layer, lower_layer, n_stages, pos, n_crp):
    upper_stage_delays = [np.random.normal(size=n_stages+1) for _ in range(upper_layer)]
    lower_stage_delays = [np.random.normal(size=n_stages+2) for _ in range(lower_layer)]
    
    x=[]
    y=[]
    for k in range(n_crp):
        upper_challenge = np.random.randint(2, size=n_stages)
        upper_feature, upper_res = XOR_PUF(upper_challenge, upper_stage_delays, upper_layer, n_stages)
        lower_challenge = np.insert(upper_challenge, pos, upper_res)
        lower_feature, lower_res = XOR_PUF(lower_challenge, lower_stage_delays, lower_layer, n_stages+1)
        x.append(upper_feature + lower_feature)
        y.append(lower_res)
    x_array = np.array(x) 
    y_array = np.array(y)    
    return x_array, y_array

x_array, y_array = iPUF(4,4,16,8,100000)
X_train, X_test, Y_train, Y_test = train_test_split(x_array, y_array, test_size = 0.2, random_state = 42)
svc = svm.SVC().fit(X_train, Y_train)
print(svc.score(X_test, Y_test))
    

0.7418
